In [ ]:
sample_set = ""
release = ""

In [ ]:
import google.auth
import gcsfs

credentials, project_id = google.auth.default()
print(credentials)
print(project_id)

gcs = gcsfs.GCSFileSystem(
      token=credentials,
      project=project_id, 
      cache_timeout=0, 
      block_size=2**18,
    )

In [ ]:
import numpy as np
import ancIBD
import allel
import malariagen_data
import dask 

dask.config.set(num_workers=4)

import pandas as pd
import itertools
from tqdm.notebook import tqdm
from numba import njit

@njit()
def convert_genotypes_to_probs(genotypes, error_rate=0.01):
    snps, samples, _ = genotypes.shape
    probs = np.zeros((snps, samples, 3), dtype=np.float64)
    
    for i in range(snps):
        for j in range(samples):
            allele1, allele2 = genotypes[i, j]
            
            if allele1 == 0 and allele2 == 0:
                probs[i, j] = [1 - error_rate, error_rate / 2, error_rate / 2]
            elif allele1 == 0 and allele2 == 1:
                probs[i, j] = [error_rate / 2, 1 - error_rate, error_rate / 2]
            elif allele1 == 1 and allele2 == 0:
                probs[i, j] = [error_rate / 2, 1 - error_rate, error_rate / 2]
            else:  # allele1 == 1 and allele2 == 1
                probs[i, j] = [error_rate / 2, error_rate / 2, 1 - error_rate]
    
    return probs

def read_ag_gmap(contig):
    if contig in {"2RL", "3RL"}:
        chrom = contig[0]
        contig_r, contig_l = f"{chrom}R", f"{chrom}L"
        df_r = read_ag_gmap(contig_r)
        df_l = read_ag_gmap(contig_l)
        max_ppos = df_r["pposition"].iloc[-1]
        max_gpos = df_r["gposition"].iloc[-1]
        df_l = df_l.iloc[1:]
        df_l["pposition"] += max_ppos
        df_l["gposition"] += max_gpos
        df = pd.concat([df_r, df_l], axis=0, ignore_index=True)
    else:
        df = pd.read_csv(gmap_dict[contig], sep="\t")
    return df
    
def ag_gmap(contig):
    ag3 = malariagen_data.Ag3()
        
    # read in the genetic map file
    df_gmap = read_ag_gmap(contig)

    # set up an array of per-base recombination rate values
    rr = np.zeros(len(ag3.genome_sequence(contig)), dtype="f8")

    # fill in the recombination rate values from the genetic map file
    for row, next_row in zip(
        itertools.islice(df_gmap.itertuples(), 0, len(df_gmap)-1), 
        itertools.islice(df_gmap.itertuples(), 1, None)):
        
        # N.B., the genetic map file is in units of cM / Mbp
        # we multiple by 1e-6 to convert to cM / bp
        rr[row.pposition-1:next_row.pposition] = row.rrate * 1e-6
    
    # compute mapping from physical to genetic position
    gmap = np.cumsum(rr)
        
    return gmap
    
def ag_p2g(contig, ppos):
    """Convert physical position (bp) to genetic position (M)."""
    gmap = ag_gmap(contig)
    gpos = gmap[ppos - 1]
    return gpos / 100 # morgans not centimorgans 


gmap_dict = {contig : f"https://raw.githubusercontent.com/anopheles-genomic-surveillance/selection-atlas/main/workflow/notebooks/ag_{contig}.gmap"
            for contig in ['2L', '2R', '3L', '3R', 'X']
}


def prepare_ancIBD_input(contig, sample_sets, sample_query, cohort_size=None):
#     import zarr
#     gt = zarr.load(f"results/crosses-gn-{contig}.zarr")
#     pos = zarr.load(f"results/pos-{contig}.zarr")
#     samples = zarr.load("results/crosses-samples.zarr")

    # load haplotypes
    ds_haps = ag3.haplotypes(region=contig, sample_sets=sample_sets, sample_query=sample_query, cohort_size=cohort_size)
    gt = allel.GenotypeArray(ds_haps['call_genotype'].values)   
    pos = ds_haps['variant_position'].values
    samples = ds_haps['sample_id'].values

    # find segregating sites
    seg_ = gt.count_alleles().is_segregating()
    gt = gt.compress(seg_, axis=0)
    pos = pos[seg_]

    # find the alternate allele frequency
    frqs = gt.count_alleles().to_frequencies()
    alt_frqs = frqs[:, 1]
    
    # convert GTs to genotype probabilities
    gp = convert_genotypes_to_probs(gt.values)

    # prepare input dictionary
    input_dict = {
        'calldata/GT':gt.values,
        'calldata/GP':gp,
        'calldata/AF': alt_frqs,
        'variants/POS':pos,
        'variants/MAP':ag_p2g(contig=contig, ppos=pos),
        'samples':samples
    }
    
    return input_dict

# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

### I need to prepare a dict containing multiple arrays for a given contig:

- calldata/GT
- calldata/GP
- genomic positions
- linkage map
- samples

Lets try and create these directly from malariagen_data. 

In [ ]:
ag3 = malariagen_data.Ag3(results_cache="results_cache/")

# sample_sets = ["AG1000G-GH", "AG1000G-BF-C", "AG1000G-TZ"]
sample_query = None

In [ ]:
from ancIBD.run import hapBLOCK_chroms

for contig in ('2RL', '3RL'):
    print("Preparing input dict")
    input_dict = prepare_ancIBD_input(contig=contig, sample_sets=sample_set, sample_query=sample_query, cohort_size=None)
    print("running ancIBD")
    df_ibd = hapBLOCK_chroms(folder_in=input_dict,
                             iids=input_dict['samples'], 
                             run_iids=[],
                             ch=contig, 
                             folder_out=f"{release}_results/ibd/{sample_set}",
                             output=False, 
                             prefix_out='', 
                             logfile=False,
                             l_model='dict',
                             e_model='haploid_gl2', 
                             h_model='FiveStateScaled', 
                             t_model='standard',
                             p_col='calldata/AF',
                             ibd_in=1,
                             ibd_out=10, 
                             ibd_jump=400,
                             min_cm=8,
                             cutoff_post=0.99,
                             max_gap=0.0075)

In [ ]:
# df_ibd = df_ibd.merge(df_samples, left_on='iid1', right_on='sample_id').merge(df_samples, left_on='iid2', right_on='sample_id')

In [ ]:
# df_ibd.groupby(['iid1', 'iid2', 'taxon_x', 'taxon_y']).agg({'size':'sum'})

In [ ]:
# from ancIBD.IO.ind_ibd import combine_all_chroms

# combine_all_chroms(chs=('2RL', '3RL'),
#                    folder_base='../results/species-ibd-',
#                    path_save='../results/species_ibd.tsv')

In [ ]:
# px.histogram(df_ibd_agg, x='size', color='taxon', template='simple_white')